In [ ]:
!pip install dash

# Imports

In [1]:
import numpy as np
import pandas as pd

import requests
from matplotlib.pyplot import plot

from dash import Dash, dcc, html, Input, Output  # pip install dash (version 2.0.0 or higher)

import json
import plotly.express as px
import plotly.io as pio

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
# pio.renderers.default = 'chrome'

# Loading Data

In [4]:
url = "https://goverlytics.p.rapidapi.com/federal-legislation/ca"

querystring = {"include_summary":"true",
               "include_text":"true",
               "include_actions":"true",
               "include_votes":"true",
               "sponsor_gender":"M",
               "limit": 25
              }

headers = {
    'x-rapidapi-host': "goverlytics.p.rapidapi.com",
    'x-rapidapi-key': "4d08b76ec0msh553b06f36bc2c52p18a08ejsne688f7dc5de5"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

# print(response.text)

In [5]:
df = pd.DataFrame(response.json()["data"])
print(df.shape)

* list of provinces and territories

In [6]:
list_provinces = ['AB', 'BC', 'MB', 'NB', 'NL', 'NT', 'NS', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT', None]

name_provinces = ["Alberta", "British Columbia", "Manitoba", "New Brunswick", "Newfoundland and Labrador",
                  "Northwest Territories", "Nova Scotia", "Nunavut", "Ontario", "Prince Edward Island",
                  "Quebec", "Saskatchewan", "Yukon", None]

In [7]:
# Passed bills per province
passed_bills_pr = dict((el,0) for el in list_provinces)
for index, row in df.iterrows():
    if row['last_major_event']['status'] == 'Royal Assent':
        passed_bills_pr[row['province_territory']] += 1

# Passed bills per province

In [8]:
new_df = df[['province_territory', 'last_major_event', 'topic']].copy()
new_df['last_status'] = 0
new_df['final_year'] = 0

for index, row in new_df.iterrows():
    new_df.loc[index, 'last_status'] = row['last_major_event']['status']
    # new_df.at[index,'last_status'] = row['last_major_event']['status']
    new_df.loc[index, 'final_year'] = row['last_major_event']['date'].split('-')[0]

new_df = new_df.drop('last_major_event', axis=1)

In [9]:
passed_bills_df = pd.DataFrame(data={"name": name_provinces, "pr_ter": list_provinces})

passed_bills_df["topic"] = None
passed_bills_df["2019"] = 0
passed_bills_df["2020"] = 0
passed_bills_df["2021"] = 0


passed_bills_df = passed_bills_df.set_index('pr_ter')

for index, row in new_df.iterrows():
    if row['last_status'] == 'Royal Assent':
        year = row['final_year']
        province = row['province_territory']
        passed_bills_df.loc[row['province_territory'], row['final_year']] += 1
    
    passed_bills_df.loc[row['province_territory'], "topic"] = row['topic']
        

passed_bills_df = passed_bills_df.reset_index()
passed_bills_df['total'] = passed_bills_df[['2019', '2020', '2021']].sum(axis=1)

# Choropleth

In [10]:
canada_province = json.load(open("../input/canada-provinces/canada_provinces.geojson", "r"))

province_id_map = {}
for feature in canada_province["features"]:
    feature['id'] = feature['properties']['cartodb_id']
    province_id_map[feature['properties']['name']] = feature['id']

province_id_map[None] = None

print(province_id_map)

passed_bills_df['id'] = passed_bills_df['name'].apply(lambda x: province_id_map[x])

passed_bills_df.head()

In [11]:
fig = px.choropleth(passed_bills_df,
                    locations='id',
                    geojson=canada_province,
                    color='total',
                    )

fig.show()

# Dash
* App layout

In [12]:
app = Dash(__name__)

In [13]:
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2019", "value": 2019},
                     {"label": "2020", "value": 2020},
                     {"label": "2021", "value": 2021}],
                 multi=False,
                 value=2019,
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_canada_map', figure={})

])

* Connect the Plotly graphs with Dash Components

In [14]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_canada_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    fig = draw_map(option_slctd)

    return container, fig

if __name__ == '__main__':
    app.run_server(debug=True)